# **Creating a CNN Model and Optimizing Using Keras Tuner**

#### **Importing Libraries**

In [24]:
#import pandas
import pandas as pd
#importing numpy
import numpy as np
# importing matplotlib
import matplotlib.pyplot as plt
#importing tensorflow
import tensorflow as tf
#importing keras from tensorflow
from tensorflow import keras
# importing Sequential from keras
from tensorflow.keras.models import Sequential
#importing Dense and Conv2D layers from keras
from tensorflow.keras.layers import Dense,Conv2D

#### **Importing Fashion MNIST Dataset**

In [2]:
df = tf.keras.datasets.fashion_mnist
# spliting the data into train and testing part
(train_df,train_labl),(test_df,test_labl) = df.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [22]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [27]:
train_df.shape

(60000, 28, 28, 1)

In [32]:
# plt.figure(figsize=(10,10));
# for i in range(25):
#     plt.subplot(5,5,i+1);
#     plt.xticks([]);
#     plt.yticks([]);
#     plt.grid(False);
#     plt.imshow(train_df[i], cmap=plt.cm.binary);
#     plt.xlabel(class_names[train_df[i]]);
# plt.show();

#### **Scaling and Reshaping Images**

In [7]:
#scaling the train data
train_df=train_df/255.0
#scaling test data
test_df = test_df/255.0

In [9]:
#rehaping the images into equal dim.
train_df = train_df.reshape(len(train_df),28,28,1)
test_df = test_df.reshape(len(test_df),28,28,1)

#### **Hypertuning the Model**

In [10]:
def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(28,28,1)),
    # adding second convolutional layer 
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

#### **Optimize Model**

In [33]:
# !pip install keras-tuner

In [14]:
#importing random search
from kerastuner import RandomSearch
#creating randomsearch object
tuner = RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 2)
# search best parameter
tuner.search(train_df,train_labl,epochs=1,validation_data=(train_df,train_labl))

Trial 2 Complete [00h 07m 24s]
val_accuracy: 0.8964833617210388

Best val_accuracy So Far: 0.8964833617210388
Total elapsed time: 00h 24m 51s


In [17]:
model=tuner.get_best_models(num_models=1)[0]
#summary of best model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 64)        1664      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 32)        51232     
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (Dense)               (None, 64)                819264    
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 872,810
Trainable params: 872,810
Non-trainable params: 0
_________________________________________________________________


#### **Fitting Data**

In [18]:
model.fit(test_df,test_labl,
          epochs=10,
          validation_split=0.1,
          initial_epoch=2)

Epoch 3/10
282/282 [==============================] - 54s 188ms/step - loss: 0.3236 - accuracy: 0.8822 - val_loss: 0.2963 - val_accuracy: 0.8940
Epoch 4/10
282/282 [==============================] - 53s 189ms/step - loss: 0.2533 - accuracy: 0.9093 - val_loss: 0.2823 - val_accuracy: 0.9020
Epoch 5/10
282/282 [==============================] - 55s 195ms/step - loss: 0.2074 - accuracy: 0.9230 - val_loss: 0.3124 - val_accuracy: 0.8900
Epoch 6/10
282/282 [==============================] - 53s 188ms/step - loss: 0.1646 - accuracy: 0.9422 - val_loss: 0.3557 - val_accuracy: 0.8870
Epoch 7/10
282/282 [==============================] - 54s 190ms/step - loss: 0.1264 - accuracy: 0.9533 - val_loss: 0.3983 - val_accuracy: 0.8900
Epoch 8/10
282/282 [==============================] - 53s 189ms/step - loss: 0.1002 - accuracy: 0.9621 - val_loss: 0.3942 - val_accuracy: 0.8980
Epoch 9/10
282/282 [==============================] - 55s 195ms/step - loss: 0.0804 - accuracy: 0.9712 - val_loss: 0.4070 - val_ac